#DATASCI W261: Machine Learning at Scale

Nick Hamlin and Tigi Thomas  
nickhamlin@gmail.com, tgthomas@berkeley.edu   
Time of Submission: 9:23 PM EST, Wednesday, Feb 10, 2016  
W261-3, Spring 2016  
Week 4 Homework

###Submission Notes:
- For each problem, we've included a summary of the question as posed in the instructions.  In many cases, we have not included the full text to keep the final submission as uncluttered as possible.  For reference, we've included a link to the original instructions in the "Useful Reference" below.
- Problem statements are listed in *italics*, while our responses are shown in plain text. 
- We've included the full output of the hadoop jobs in our responses so that counter results are shown.  However, these don't always render nicely into PDF form.  In these situations, please reference [the complete rendered notebook on Github](https://github.com/nickhamlin/mids_261_homework/blob/master/HW3/MIDS-W261-2015-HWK-Week03-Hamlin-Thomas.ipynb)

###Useful References:
- **[Original Assignment Instructions](https://www.dropbox.com/sh/m0nxsf4vs5cyrp2/AACYOZQ3hRyGtHoPt33ny_Pza/HW4-Questions.txt?dl=0)**
- [Most frequent word example in mrjob](http://nbviewer.jupyter.org/urls/dl.dropbox.com/s/nd2wow1t3y77jqk/MrjobMostUsedWord.ipynb)


##HW4.0.  
*What is MrJob? How is it different to Hadoop MapReduce?*  

*What are the mappint_init, mapper_final(), combiner_final(), reducer_final() methods? When are they called?*  



##HW4.1

*What is serialization in the context of MrJob or Hadoop?*  

*When it used in these frameworks?*  

*What is the default serialization mode for input and outputs for MrJob?*  

## HW 4.2

Recall the Microsoft logfiles data from the async lecture. The logfiles are described are located at:

https://kdd.ics.uci.edu/databases/msweb/msweb.html
http://archive.ics.uci.edu/ml/machine-learning-databases/anonymous/

This dataset records which areas (Vroots) of www.microsoft.com each user visited in a one-week timeframe in Feburary 1998.

 Here, you must preprocess the data on a single node (i.e., not on a cluster of nodes) from the format:

C,"10001",10001   #Visitor id 10001  
V,1000,1          #Visit by Visitor 10001 to page id 1000  
V,1001,1          #Visit by Visitor 10001 to page id 1001  
V,1002,1          #Visit by Visitor 10001 to page id 1002  
C,"10002",10002   #Visitor id 10001  
V  
Note: #denotes comments
to the format:

V,1000,1,C, 10001  
V,1001,1,C, 10001  
V,1002,1,C, 10001  

Write the python code to accomplish this.

In [28]:
%%writefile convert_msdata.py

from csv import reader
with open('anonymous-msweb.data','rb') as f:
    data=f.readlines()
    
for i in reader(data):
    if i[0]=='C':
        visitor_id=i[1]
        continue
    if i[0]=='V':
        print i[0]+','+i[1]+','+i[2]+',C,'+visitor_id

Overwriting convert_msdata.py


In [127]:
%%writefile create_urls.py

from csv import reader
with open('anonymous-msweb.data','rb') as f:
    data=f.readlines()
    
for i in reader(data):
    if i[0]=='A':
        print i[1]+','+i[3]+','+i[4]

Writing create_urls.py


In [128]:
#Make files executable, convert data, view some results
#!chmod +x convert_msdata.py create_urls.py
!python convert_msdata.py > clean_msdata.txt
!cat clean_msdata.txt | head -10
!python create_urls.py > ms_urls.txt
!cat ms_urls.txt | head -10

V,1000,1,C,10001
V,1001,1,C,10001
V,1002,1,C,10001
V,1001,1,C,10002
V,1003,1,C,10002
V,1001,1,C,10003
V,1003,1,C,10003
V,1004,1,C,10003
V,1005,1,C,10004
V,1006,1,C,10005
cat: stdout: Broken pipe
1287,International AutoRoute,/autoroute
1288,library,/library
1289,Master Chef Product Information,/masterchef
1297,Central America,/centroam
1215,For Developers Only Info,/developer
1279,Multimedia Golf,/msgolf
1239,Microsoft Consulting,/msconsult
1282,home,/home
1251,Reference Support,/referencesupport
1121,Microsoft Magazine,/magazine


## HW 4.3
*Find the 5 most frequently visited pages using MrJob from the output of 4.2 (i.e., transfromed log file).*

In [18]:
%load_ext autoreload
%autoreload 2

In [125]:
%%writefile top_pages.py
"""
This program will take a CSV data file and output tab-seperated lines of

    Vroot -> number of visits

To run:

    python top_pages.py anonymous-msweb.data

To store output:

    python top_pages.py anonymous-msweb.data > top_pages.out
"""
from mrjob.job import MRJob
import csv

def csv_readline(line):
    """Given a string CSV line, return a list of strings."""
    for row in csv.reader([line]):
        return row

class TopPages(MRJob):
    
# Normally, we'd use the shuffle to do the sort.  However, the bug
# in comparitors when running local MRJobs makes this an untenable solution
# so we'll settle for doing the sort in the second-stage reducer instead

#     def jobconf(self):
#         orig_jobconf = super(TopPages, self).jobconf()        
#         custom_jobconf = {  #key value pairs
#             'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
#             'mapred.text.key.comparator.options': '-k1,1nr',
#             'mapred.reduce.tasks': '1',
#         }
#         combined_jobconf = orig_jobconf
#         combined_jobconf.update(custom_jobconf)
#         self.jobconf = combined_jobconf
#         return combined_jobconf

    def mapper_extract_views(self, line_no, line):
        """Extracts the Vroot that was visited"""
        cell = csv_readline(line)
        if cell[0] == 'V':
            yield cell[1],1

    def reducer_sum_views(self, vroot, visit_counts):
        """Sumarizes the visit counts by adding them together,yield the results"""
        
        total = sum(i for i in visit_counts)
        yield None,(total, vroot)
        
        
    # discard the key; it is just None
    def reducer_find_top_views(self,_, word_count_pairs):
        # each item of word_count_pairs is (count, word),
        # so yielding one results in key=counts, value=word

        output=sorted(word_count_pairs)[-5:]
        output.reverse()
        for i in output:
            yield (i[1],i[0])
        
        
    def steps(self):  #pipeline of Map-Reduce jobs
        return [
            self.mr(mapper=self.mapper_extract_views,       # STEP 1: view count step
                    reducer=self.reducer_sum_views) ,
            self.mr(reducer=self.reducer_find_top_views) # Step 2: sort and return top 5 results
        ]
        
if __name__ == '__main__':
    TopPages.run()

Overwriting top_pages.py


In [126]:
from top_pages import TopPages
import csv

mr_job = TopPages(args=['clean_msdata.txt'])
with mr_job.make_runner() as runner:
    runner.run()
    for line in runner.stream_output():
        print mr_job.parse_output_line(line)

('1008', 10836)
('1034', 9383)
('1004', 8463)
('1018', 5330)
('1017', 5108)


In [112]:
test=[('1','a'),('3','f'),('2','s'),('87','ab'),('11','a')]
output=[(int(i[0]),i[1]) for i in test]
output.sort()
output.reverse()
print output

[(87, 'ab'), (11, 'a'), (3, 'f'), (2, 's'), (1, 'a')]


In [33]:
!chmod +x top_pages.py

In [ ]:
!python mr_wc.py mr_wc.py

In [166]:
from collections import Counter
from operator import itemgetter
test=Counter()
visits=[1,1,2,2,2,5,5,5,4]
for i in visits:
    test[i]+=1
    
output= max(test.iteritems(), key=itemgetter(1))[0]

print test
print output
test[output]

Counter({2: 3, 5: 3, 1: 2, 4: 1})
2


3

## HW 4.4

*Find the most frequent visitor of each page using MrJob and the output of 4.2  (i.e., transfromed log file). In this output please include the webpage URL, webpageID and Visitor ID.*

In [178]:

print url_dict[1002]

/athome


In [28]:
!pwd

/Users/nicholashamlin/Documents/Grad School/261 - Machine Learning at Scale/mids_261_homework/HW4


In [31]:
%%writefile freq_visitor.py

import csv
from collections import Counter
from operator import itemgetter

from mrjob.job import MRJob
from mrjob.step import MRStep


def csv_readline(line):
    """Given a string CSV line, return a list of strings."""
    for row in csv.reader([line]):
        return row

class FreqVisitor(MRJob):
    
    def jobconf(self):
        orig_jobconf = super(FreqVisitor, self).jobconf()        
        custom_jobconf = {'upload_files': '/Users/nicholashamlin/Documents/Grad School/261 - Machine Learning at Scale/mids_261_homework/HW4/ms_urls.txt'}
        combined_jobconf = orig_jobconf
        combined_jobconf.update(custom_jobconf)
        self.jobconf = combined_jobconf
        return combined_jobconf

    def mapper_extract_views(self, line_no, line):
        """Extracts the visitor id and the vroot that was visited"""
        cell = csv_readline(line)
        if cell[0] == 'V':
            yield cell[4],cell[1]
    
    def reducer_load_urls(self):
        with open('ms_urls.txt','rb') as f:
            urls=csv.reader(f.readlines())
        self.url_dict={}
        for i in urls:
            self.url_dict[int(i[0])]=i[2]

    def reducer_sum_views_by_visitor(self, visitor, vroots):
        """Summarizes page counts for each visitor, 
        yields one record per visitor with the page containing 
        the most views by that visitor"""
        pages=Counter()
        for i in vroots:
            pages[i]+=1
        output= max(pages.iteritems(), key=itemgetter(1))[0]
        yield (visitor,output),pages[output]#,url_dict[output]
   
    def steps(self):
        return [MRStep(mapper=self.mapper_extract_views,
                        #reducer_init=self.reducer_load_urls,
                        reducer=self.reducer_sum_views_by_visitor)]
        
if __name__ == '__main__':
    FreqVisitor.run()

Overwriting freq_visitor.py


In [32]:
from freq_visitor import FreqVisitor
import csv

mr_job = FreqVisitor(args=['clean_msdata.txt'])
with mr_job.make_runner() as runner:
    runner.run()
    for line in runner.stream_output():
        print mr_job.parse_output_line(line)

IOError: [Errno 2] No such file or directory: 'ms_urls.txt'

## HW 4.5


Here you will use a different dataset consisting of word-frequency distributions 
for 1,000 Twitter users. These Twitter users use language in very different ways,
and were classified by hand according to the criteria:

0: Human, where only basic human-human communication is observed.

1: Cyborg, where language is primarily borrowed from other sources
(e.g., jobs listings, classifieds postings, advertisements, etc...).

2: Robot, where language is formulaically derived from unrelated sources
(e.g., weather/seismology, police/fire event logs, etc...).

3: Spammer, where language is replicated to high multiplicity
(e.g., celebrity obsessions, personal promotion, etc... )

Check out the preprints of our recent research,
which spawned this dataset:

http://arxiv.org/abs/1505.04342
http://arxiv.org/abs/1508.01843

The main data lie in the accompanying file:

topUsers_Apr-Jul_2014_1000-words.txt

and are of the form:

USERID,CODE,TOTAL,WORD1_COUNT,WORD2_COUNT,...
.
.

where

USERID = unique user identifier
CODE = 0/1/2/3 class code
TOTAL = sum of the word counts

Using this data, you will implement a 1000-dimensional K-means algorithm in MrJob on the users
by their 1000-dimensional word stripes/vectors using several 
centroid initializations and values of K.

Note that each "point" is a user as represented by 1000 words, and that
word-frequency distributions are generally heavy-tailed power-laws
(often called Zipf distributions), and are very rare in the larger class
of discrete, random distributions. For each user you will have to normalize
by its "TOTAL" column. Try several parameterizations and initializations:

(A) K=4 uniform random centroid-distributions over the 1000 words
(B) K=2 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution 
(C) K=4 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution 
(D) K=4 "trained" centroids, determined by the sums across the classes.

and iterate until a threshold (try 0.001) is reached.
After convergence, print out a summary of the classes present in each cluster.
In particular, report the composition as measured by the total
portion of each class type (0-3) contained in each cluster,
and discuss your findings and any differences in outcomes across parts A-D.

Note that you do not have to compute the aggregated distribution or the 
class-aggregated distributions, which are rows in the auxiliary file:

topUsers_Apr-Jul_2014_1000-words_summaries.txt

###End of Submission